<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/preprocessing/comment_policy_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List, Dict
def _have_langchain_and_ollama():
    try:
        from langchain_ollama import ChatOllama
        from langchain.prompts import ChatPromptTemplate
        return True
    except Exception:
        return False

def _mk_model(model_name: str = "qwen2.5-coder:7b", temperature: float = 0.0):
    from langchain_ollama import ChatOllama
    return ChatOllama(model=model_name, temperature=temperature)

In [ ]:
BATCH_PROMPT = """You are an LLM that decides whether SOURCE CODE COMMENTS should be KEPT in a fine-tuning dataset for a coding model.
For each item, return EXACTLY one token: KEEP or DROP. Output one line per item, in the same order.

Guidelines (prioritize KEEP when it improves learning):
- KEEP comments that explain algorithmic intent, data structures, invariants, pre/post-conditions, proofs/specifications, security reasoning, or nontrivial usage examples.
- KEEP docstrings/API docs (Args/Parameters/Returns), and comments mapping code to external specs (e.g., Cryptol/SAW/Lean proofs).
- DROP license headers, copyright notices, autogenerated banners, vendored/minified blobs, dependency lock dumps, or giant changelogs.
- DROP comments that only restate the code without insight (e.g., "increment i").
- Short inline hints are usually helpful → KEEP.
- Very long boilerplate with legal language → DROP.

You will receive N items in this format:

ITEM i
FILE: <path>
COMMENT:
---
<text>
---
CODE CONTEXT:
---
<code>
---

Respond with N lines, each either KEEP or DROP, no numbering.
"""

In [ ]:
def _fallback_batch(items: List[Dict]) -> List[bool]:
    outs = []
    for it in items:
        txt = it["comment_text"]
        lower = txt.lower()
        if any(k in lower for k in ["copyright", "license", "warranty", "apache", "mit", "bsd", "gnu"]):
            outs.append(False); continue
        if len(txt) > 1000 and "http" in lower and "@" in txt:
            outs.append(False); continue
        if any(k in lower for k in ["args", "parameters", "returns", "example", "usage", "invariant", "precondition", "postcondition", "proof", "spec"]):
            outs.append(True); continue
        outs.append(len(txt) < 500)
    return outs

def decide_keep_drop_batch(items: List[Dict], model_name: str = "qwen2.5-coder:7b") -> List[bool]:
    """
    items: List[{'comment_text': str, 'file_path': str, 'code_context': str}]
    returns: List[bool] KEEP=True, DROP=False for each item, in order
    """
    if not _have_langchain_and_ollama():
        return _fallback_batch(items)

    from langchain.prompts import ChatPromptTemplate
    model = _mk_model(model_name=model_name, temperature=0.0)
    parts = [BATCH_PROMPT, "=== ITEMS START ==="]
    for i, it in enumerate(items):
        parts.append(f"ITEM {i}")
        parts.append(f"FILE: {it.get('file_path','')}")
        parts.append("COMMENT:\\n---\\n" + it.get('comment_text','')[:4000] + "\\n---")
        parts.append("CODE CONTEXT:\\n---\\n" + it.get('code_context','')[:4000] + "\\n---")
    parts.append("=== ITEMS END ===\\nRespond now with exactly one line per item: KEEP or DROP.")
    prompt_text = "\\n".join(parts)

    resp = model.invoke([("human", prompt_text)])
    text = (resp.content or "").strip().upper()
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    outs = []
    for ln in lines[:len(items)]:
        if "KEEP" in ln and "DROP" not in ln:
            outs.append(True)
        elif "DROP" in ln and "KEEP" not in ln:
            outs.append(False)
        else:
            outs.append(len(items[len(outs)]["comment_text"]) < 500)
    while len(outs) < len(items):
        outs.append(len(items[len(outs)]["comment_text"]) < 500)
    return outs